In [5]:
import pandas as pd
import numpy as np
import re

print("=== 《与星共舞》数据清洗脚本 - 修正版本 ===\n")

=== 《与星共舞》数据清洗脚本 - 修正版本 ===



In [6]:
# ===================== 统一文件命名常量区 =====================
# 原始数据读取文件名（可在此统一修改）
RAW_DATA_FILE = '2026_MCM_Problem_C_Data.csv'
# 清洗后数据保存文件名（可在此统一修改）
CLEANED_DATA_FILE = '1_cleaned_data.csv'
# ==============================================================

In [7]:
# 步骤1：加载数据
print("步骤1：加载数据...")
try:
    df = pd.read_csv(RAW_DATA_FILE, encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(RAW_DATA_FILE, encoding='latin-1')
        print("使用latin-1编码读取文件")
    except Exception as e:
        print(f"读取文件失败: {e}")
        exit()

print(f"原始数据形状: {df.shape}")
print(df.head())
print("\n" + "="*50 + "\n")

步骤1：加载数据...
原始数据形状: (421, 53)
      celebrity_name     ballroom_partner celebrity_industry  \
0      John O'Hurley  Charlotte Jorgensen      Actor/Actress   
1       Kelly Monaco            Alec Mazo      Actor/Actress   
2  Evander Holyfield      Edyta Sliwinska            Athlete   
3      Rachel Hunter     Jonathan Roberts              Model   
4      Joey McIntyre      Ashly DelGrosso      Singer/Rapper   

  celebrity_homestate celebrity_homecountry/region  \
0               Maine                United States   
1        Pennsylvania                United States   
2             Alabama                United States   
3                 NaN                  New Zealand   
4       Massachusetts                United States   

   celebrity_age_during_season  season            results  placement  \
0                           50       1          2nd Place          2   
1                           29       1          1st Place          1   
2                           42       1  Elim

In [8]:
# 步骤2：预处理原始数据
print("步骤2：预处理原始数据...")
initial_len = len(df)
df = df.drop_duplicates().reset_index(drop=True)
print(f"删除了 {initial_len - len(df)} 行重复数据")

key_columns = ['celebrity_name', 'season']
if all(col in df.columns for col in key_columns):
    before_len = len(df)
    df = df.dropna(subset=key_columns)
    print(f"删除了 {before_len - len(df)} 行关键列缺失的数据")

print(f"预处理后数据形状: {df.shape}")
print("\n" + "="*50 + "\n")

步骤2：预处理原始数据...
删除了 0 行重复数据
删除了 0 行关键列缺失的数据
预处理后数据形状: (421, 53)




In [9]:
# 步骤3：转换数据为长格式
print("步骤3：转换为长格式...")

id_vars = [col for col in df.columns if 'week' not in col.lower()]
value_vars = [col for col in df.columns if 'week' in col.lower() and 'judge' in col.lower()]

df_long = pd.melt(df,
                  id_vars=id_vars,
                  value_vars=value_vars,
                  var_name='week_judge',
                  value_name='judge_score')

df_long['week'] = df_long['week_judge'].str.extract(r'week(\d+)').astype(float)
df_long['judge_number'] = df_long['week_judge'].str.extract(r'judge(\d+)').astype(float)
df_long = df_long.drop(columns=['week_judge'])

print(f"长格式数据形状: {df_long.shape}")
print("\n" + "="*50 + "\n")

步骤3：转换为长格式...
长格式数据形状: (18524, 12)




In [10]:
# 步骤4：清洗评委分数并标记淘汰状态
print("步骤4：清洗评委分数并标记淘汰状态...")

df_long['judge_score'] = pd.to_numeric(df_long['judge_score'], errors='coerce')

# 更精确地标记淘汰：如果某周后所有分数都是0或NaN，则该选手在此周被淘汰
df_long = df_long.sort_values(by=['celebrity_name', 'season', 'week']).reset_index(drop=True)

# 创建一个函数来标记淘汰
def mark_elimination(group):
    """标记选手被淘汰的周次"""
    # 找到第一个无效分数（0或NaN）开始的周次
    valid_scores = group[(group['judge_score'].notna()) & (group['judge_score'] > 0)]
    if len(valid_scores) == 0:
        # 如果没有有效分数，则从第一周就被淘汰
        group['was_eliminated'] = True
    else:
        last_valid_week = valid_scores['week'].max()
        group['was_eliminated'] = group['week'] > last_valid_week
    return group

# 应用淘汰标记
df_long = df_long.groupby(['celebrity_name', 'season']).apply(mark_elimination).reset_index(drop=True)

print("已处理淘汰状态标记")
print("\n" + "="*50 + "\n")

步骤4：清洗评委分数并标记淘汰状态...
已处理淘汰状态标记




C:\Users\harry_pv15g7u\AppData\Local\Temp\ipykernel_47928\1685931183.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_long = df_long.groupby(['celebrity_name', 'season']).apply(mark_elimination).reset_index(drop=True)


In [11]:
# 步骤5：诊断评委人数一致性（在聚合之前进行）
print("步骤5：诊断评委人数一致性...")

# 只考虑当周未被淘汰的选手
active_players = df_long[df_long['was_eliminated'] == False]

# 计算每位存活选手在该周获得的实际打分次数
judge_count_check = active_players.groupby(['season', 'week', 'celebrity_name'])['judge_score'].count().reset_index()
judge_count_variation = judge_count_check.groupby(['season', 'week'])['judge_score'].nunique()
weeks_with_varying_judges = judge_count_variation[judge_count_variation > 1]

if len(weeks_with_varying_judges) > 0:
    print(f"发现 {len(weeks_with_varying_judges)} 个周次存在评委人数不一致的情况。")
    print("具体示例如下：")
    
    # 打印前几个示例的详细情况
    for (season, week), _ in list(weeks_with_varying_judges.head(3).items()):
        print(f"\n--- 第 {int(season)} 季, 第 {int(week)} 周 ---")
        week_data = judge_count_check[(judge_count_check['season'] == season) & (judge_count_check['week'] == week)]
        print("各选手评委人数统计:")
        print(week_data[['celebrity_name', 'judge_score']].sort_values('judge_score', ascending=False))
        
        # 显示该周详细的评委打分情况
        detailed_scores = active_players[(active_players['season'] == season) & (active_players['week'] == week)]
        print("\n详细的评委打分情况:")
        pivot_table = detailed_scores.pivot_table(
            index='celebrity_name', 
            columns='judge_number', 
            values='judge_score', 
            aggfunc='first'
        )
        print(pivot_table)
else:
    print("所有周次中，每位存活选手的评委人数均一致。")

print("\n" + "="*50 + "\n")

步骤5：诊断评委人数一致性...
所有周次中，每位存活选手的评委人数均一致。




In [12]:
# 步骤6：创建每周聚合特征
print("步骤6：创建每周特征...")

# 按选手、赛季和周次分组计算统计量
weekly_agg = df_long.groupby(['celebrity_name', 'season', 'week']).agg(
    weekly_avg_score=('judge_score', 'mean'),
    weekly_total_score=('judge_score', 'sum'),
    num_judges=('judge_score', 'count'),
    was_eliminated=('was_eliminated', 'first')  # 从df_long中获取淘汰标记
).reset_index()

# 计算排名（只对未淘汰选手进行排名）
weekly_agg['weekly_rank_by_avg'] = weekly_agg.groupby(['season', 'week'])['weekly_avg_score'].rank(
    method='min', ascending=False)
weekly_agg['weekly_rank_by_total'] = weekly_agg.groupby(['season', 'week'])['weekly_total_score'].rank(
    method='min', ascending=False)

# 计算百分比
weekly_totals = weekly_agg.groupby(['season', 'week'])['weekly_total_score'].transform('sum')
weekly_agg['weekly_score_percentage'] = (weekly_agg['weekly_total_score'] / weekly_totals) * 100

# 合并其他信息
if 'results' in df.columns:
    result_info = df[['celebrity_name', 'season', 'results', 'placement']].drop_duplicates()
    weekly_agg = weekly_agg.merge(result_info, on=['celebrity_name', 'season'], how='left')

print("每周特征创建完成")
print(f"最终数据形状: {weekly_agg.shape}")
print("\n最终数据列名:")
print(weekly_agg.columns.tolist())
print("\n" + "="*50 + "\n")

步骤6：创建每周特征...
每周特征创建完成
最终数据形状: (4631, 12)

最终数据列名:
['celebrity_name', 'season', 'week', 'weekly_avg_score', 'weekly_total_score', 'num_judges', 'was_eliminated', 'weekly_rank_by_avg', 'weekly_rank_by_total', 'weekly_score_percentage', 'results', 'placement']




In [13]:
# 步骤7：保存结果
print("步骤7：保存清洗后的数据...")
weekly_agg.to_csv(CLEANED_DATA_FILE, index=False)
print(f"数据已保存为 '{CLEANED_DATA_FILE}'")

print("\n=== 数据清洗完成! ===")

# 显示最终数据样例
print("\n清洗后数据前10行样例:")
print(weekly_agg.head(10))

步骤7：保存清洗后的数据...
数据已保存为 '1_cleaned_data.csv'

=== 数据清洗完成! ===

清洗后数据前10行样例:
  celebrity_name  season  week  weekly_avg_score  weekly_total_score  \
0      AJ McLean      29   1.0          6.000000             18.0000   
1      AJ McLean      29   2.0          6.333333             19.0000   
2      AJ McLean      29   3.0          7.000000             21.0000   
3      AJ McLean      29   4.0          8.000000             24.0000   
4      AJ McLean      29   5.0          8.000000             24.0000   
5      AJ McLean      29   6.0          9.000000             27.0000   
6      AJ McLean      29   7.0          8.666667             26.0000   
7      AJ McLean      29   8.0          9.000000             27.0000   
8      AJ McLean      29   9.0          8.333267             24.9998   
9      AJ McLean      29  10.0          0.000000              0.0000   

   num_judges  was_eliminated  weekly_rank_by_avg  weekly_rank_by_total  \
0           3           False                 6.0        